In [77]:
import numpy as np

In [78]:
class VectorOfMatrices:
    def __init__(self, matrices):
        self.matrices = matrices
     
    def __mul__(self, other):
        # https://doc.sagemath.org/html/en/reference/matrices/sage/matrix/matrix_dense.html
        if not isinstance(other, sage.matrix.matrix_dense.Matrix_dense):
            raise TypeError("We only multiply by a matrix my dear")

        if other.ncols() != len(self.matrices):
            raise ValueError("Matrix dimensions do not match")

        # multiply them by hand
        result = [sum(other[i, j] * self.matrices[j] for j in range(len(self.matrices))) for i in range(other.nrows())]
        # gives back a list
        return result

    def __repr__(self):
        return f"VectorOfMatrices(\n{self.matrices})"
    
    __rmul__ = __mul__

In [99]:
class Instance:
    def __init__(self, n, m, p):
        self.n = n
        self.m = m
        self.size = p
        self.field = GF(p)
        
        self.F = []
        self.S = np.zeros((self.n,self.n), dtype=int)
        self.T = np.zeros((self.m, self.m), dtype=int)
        
        self.P = []

    def _random_symmetric_matrix(self):
        # get a random matrix
        A = random_matrix(self.field, self.n, self.n)
        # and make it symmetric
        return A + A.transpose()

    def _random_polynomial(self):
        return self._random_symmetric_matrix(), random_vector(self.field, self.n), self.field.random_element()

    def generate_private(self):
        self.T = matrix(GL(self.m, self.field).random_element())
        self.S = matrix(GL(self.n, self.field).random_element())
        if self.F == []:
            # print(f"Generating random polynomials over a finite field with {self.size} elements")
            self.F = [[],[],[]]
            for i in range(m):
                A, B, C = self._random_polynomial()
                self.F[0].append(A)
                self.F[1].append(B)
                self.F[2].append(C)
    
    def show_private(self):
        print(f"The input transformation S\n{self.S}")
        print(f"The output transformation T\n{self.T}")
        print("The central map f^{(i)}(x) = x^T Ax + bx + c")
        for i in range(m):
            print(f"F[{i}]:Matrix\n{self.F[0][i]}, Vector {self.F[1][i]}, Scalar {self.F[2][i]}\n---------------------------------------------")

    def generate_public(self):
        if self.P == []:
            self.P = [[],[],[]]
            v = VectorOfMatrices([self.S.transpose()*A*self.S for A in self.F[0]])
            self.P = [self.T*v, self.T*matrix([B*self.S for B in self.F[1]]), self.T*vector(self.F[2])]

    def show_public(self):
        print("The public set of polynomials p^{(i)}(x) = sum_{j=1}^m t_{i,j} f^{(j)}(Sx)")
        for i in range(m):
            print(f"P[{i}]: Matrix\n{self.P[0][i]}, Vector {self.P[1][i]}, Scalar {self.P[2][i]}\n---------------------------------------------")

    def model(self):
        R  = PolynomialRing(self.field, self.n^2 + self.m^2, 'x')
        R.inject_variables(verbose=False)
        gens = R.gens()

        # first n^2 elements of R2
        S_ = matrix(R, n, n, gens[:n^2])

        # we get the last m^2 variables of R2
        T_ = matrix(R, m, m, gens[-m^2:])

        v_ = VectorOfMatrices([S_.transpose()*A*S_ for A in self.F[0]])
        auxiliary = [[],[],[]]
        auxiliary[0] = T_*v_
        
        auxiliary[1] = T_*matrix([B*S_ for B in self.F[1]])
        
        auxiliary[2] = T_*vector(self.F[2])
        
        # add the field equations by hand
        system = [x^self.size - x for x in gens]
        for i in range(m):
            system += (auxiliary[0][i] - self.P[0][i]).list()
            system += (auxiliary[1][i] - self.P[1][i]).list()
            system += [auxiliary[2][i] - self.P[2][i]]
        
        # the list(set(.)) removes the duplicates
        return R.ideal(list(set(system)))

    # given a system of polynomials, check if it correctly corresponds
    # to the actual matrices it was created from
    #
    # ideal is an ideal of multivariate polynomial ring defined above
    # S and T are matrices
    def is_correct(self, ideal):
        point = self.S.list()+self.T.list()
    
        R = ideal.ring()
        gens = R.gens()
        # make an appropriate dict
        substitution_dict = {gens[i]: point[i] for i in range(len(point))}
        
        return all(f.subs(substitution_dict) == 0 for f in ideal.gens())

In [100]:
p = 2
n = 4
m = 3

inst = Instance(n, m, p)
inst.generate_private()
# inst.show_private()

inst.generate_public()
# inst.show_public()

I = inst.model()
# print(I)
# I.variety()
inst.is_correct(I)

True